In [ ]:
import requests
import pandas as pd
from datetime import datetime, timezone, date
from dateutil import parser

In [ ]:
url = 'https://backend.metruyencv.com/api/books?filter%5Bgender%5D=1&filter%5Bkind%5D=1&filter%5Bstate%5D=published&filter%5Bstatus%5D=2&include=author%2Cgenres%2Ccreator&limit=20&sort=-new_chap_at'

page = 1
data_list = []
update_day = datetime(2024, 6, 3, tzinfo=timezone.utc)  # Update day set to 3/6/2024

while True:
    page_url = url + f"&page={page}"
    response = requests.get(page_url)

    if response.status_code == 200:
        data = response.json()
        books = data['data']

        # Kiểm tra nếu không còn dữ liệu, thoát khỏi vòng lặp
        if not books:
            print('Done!')
            break
        filtered_books = [book for book in books if parser.isoparse(book['new_chap_at']) >= update_day] # Lấy bắt đầu từ lần lấy trước đó
        data_list.extend(filtered_books)

        # data_list.extend(books) # Lấy full
        page += 1
    else:
        print("Yêu cầu không thành công. Mã trạng thái:", response.status_code)
        break

Done!


In [ ]:
# Tạo DataFrame từ danh sách dữ liệu
book_list = pd.DataFrame(data_list, columns=['id', 'name', 'author', 'chapter_count', 'chapter_per_week', 'comment_count', 'creator', 'genres', 'link',
                                      'review_count', 'review_score', 'view_count', 'vote_count', 'word_count', 'created_at', 'new_chap_at', 'synopsis'])


In [ ]:
book_list.head(2)

,id,name,author,chapter_count,chapter_per_week,comment_count,creator,genres,link,review_count,review_score,view_count,vote_count,word_count,created_at,new_chap_at,synopsis
0,111871,Toàn Dân Hải Đảo: Ta Hải Đảo Dĩ Nhiên Sống Rồi!,"{'id': 6250, 'name': 'Khoái Phù Ngã Khởi Lai',...",785,0,267,"{'id': 1222631, 'name': 'Thích Ăn Kem', 'avata...","[{'id': 3, 'name': 'Huyền Huyễn', 'object_type...",https://metruyencv.com/truyen/toan-dan-hai-dao...,16,4.552,783166,373,1256221,2022-04-03T11:44:56.000000Z,2024-05-13T09:43:41.000000Z,« B.faloo mạng tiểu thuyết độc nhất vô nhị ký ...
1,125963,"Dị Tộc Xâm Lấn, Ta Dẫn Đầu Toàn Nhân Tộc Chuyể...","{'id': 13288, 'name': 'Khoa Học Cấm Khu', 'ava...",112,0,14,"{'id': 1222631, 'name': 'Thích Ăn Kem', 'avata...","[{'id': 6, 'name': 'Đô Thị', 'object_type': 'G...",https://metruyencv.com/truyen/di-toc-xam-lan-t...,0,0.000,4610,0,192899,2024-04-19T09:03:00.000000Z,2024-05-13T08:48:37.000000Z,"Vạn tộc xâm lấn, Nhân tộc tràn ngập nguy hiểm...."


In [ ]:
def transform_df (df):
  ######----AUTHOR---######

  # Thay đổi tên cột và loại bỏ giá trị ban đầu
  df['author_id'] = df['author'].apply(lambda x: x['id'])
  df['author'] = df['author'].apply(lambda x: x['name'])

  # Tạo DataFrame mới cho tác giả
  author_df = pd.DataFrame()
  author_df['author_id'] = df['author_id']
  author_df['author_name'] = df['author']
  author_df = author_df.drop_duplicates()
  author_df = author_df.sort_values('author_id').reset_index(drop=True)

  # Xóa cột author
  df = df.drop('author', axis=1)

  ######----TRANSLATOR---######

  # Thay đổi tên cột và loại bỏ giá trị ban đầu
  df['translator_id'] = df['creator'].apply(lambda x: x['id'])
  df['creator'] = df['creator'].apply(lambda x: x['name'])

  # Tạo DataFrame mới cho dịch giả
  translator_df = pd.DataFrame()
  translator_df['translator_id'] = df['translator_id']
  translator_df['translator_name'] = df['creator']
  translator_df = translator_df.drop_duplicates()
  translator_df = translator_df.sort_values('translator_id').reset_index(drop=True)

  # Xóa cột creator
  df = df.drop('creator', axis=1)

  ######----GENRES---######

  # Thay đổi tên cột và loại bỏ giá trị ban đầu
  df['genre_id'] = df['genres'].apply(lambda x: x[0]['id'])
  df['genres'] = df['genres'].apply(lambda x: x[0]['name'])

  # Tạo DataFrame mới cho thể loại
  genre_df = pd.DataFrame()
  genre_df['genre_id'] = df['genre_id']
  genre_df['genre_name'] = df['genres']
  genre_df = genre_df.drop_duplicates()
  genre_df = genre_df.sort_values('genre_id').reset_index(drop=True)

  # Xóa cột genres
  df = df.drop('genres', axis=1)

  ######----SYNOPSIS---######

  # Bỏ các ký tự \n, \t trong synopsis
  df['synopsis'] = df['synopsis'].str.replace('\n', '').str.replace('\t', '')
  # Đổi tên cột synopsis
  df = df.rename(columns={'synopsis': 'description'})

  ######----DATETIME---######

  # Chuyển định dạng và kiểu dữ liệu của created_at và new_chap_at
  df['created_at'] = pd.to_datetime(df['created_at'])
  df['created_at'] = df['created_at'].dt.strftime("%d-%m-%Y")
  df['created_at'] = pd.to_datetime(df['created_at'], format="%d-%m-%Y")

  df['new_chap_at'] = pd.to_datetime(df['new_chap_at'])
  df['new_chap_at'] = df['new_chap_at'].dt.strftime("%d-%m-%Y")
  df['new_chap_at'] = pd.to_datetime(df['new_chap_at'], format="%d-%m-%Y")

  # Đổi tên cột new_chap_at
  df = df.rename(columns={'new_chap_at': 'completed_at'})

  ######----RETURN---######

  return df, author_df, translator_df, genre_df


In [ ]:
fact_book, dim_author, dim_translator, dim_genre = transform_df (book_list)

In [ ]:
fact_book.head(2)

,id,name,chapter_count,chapter_per_week,comment_count,link,review_count,review_score,view_count,vote_count,word_count,created_at,completed_at,description,author_id,translator_id,genre_id
0,111871,Toàn Dân Hải Đảo: Ta Hải Đảo Dĩ Nhiên Sống Rồi!,785,0,267,https://metruyencv.com/truyen/toan-dan-hai-dao...,16,4.552,783166,373,1256221,2022-04-03,2024-05-13,« B.faloo mạng tiểu thuyết độc nhất vô nhị ký ...,6250,1222631,3
1,125963,"Dị Tộc Xâm Lấn, Ta Dẫn Đầu Toàn Nhân Tộc Chuyể...",112,0,14,https://metruyencv.com/truyen/di-toc-xam-lan-t...,0,0.000,4610,0,192899,2024-04-19,2024-05-13,"Vạn tộc xâm lấn, Nhân tộc tràn ngập nguy hiểm....",13288,1222631,6


In [ ]:
fact_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                49 non-null     int64         
 1   name              49 non-null     object        
 2   chapter_count     49 non-null     int64         
 3   chapter_per_week  49 non-null     int64         
 4   comment_count     49 non-null     int64         
 5   link              49 non-null     object        
 6   review_count      49 non-null     int64         
 7   review_score      49 non-null     object        
 8   view_count        49 non-null     int64         
 9   vote_count        49 non-null     int64         
 10  word_count        49 non-null     int64         
 11  created_at        49 non-null     datetime64[ns]
 12  completed_at      49 non-null     datetime64[ns]
 13  description       49 non-null     object        
 14  author_id         49 non-nul

In [ ]:
dim_author.head()

,author_id,author_name
0,92,Trần Ái Đình
1,240,Miêu Hình Bóng
2,914,Văn Sao Công
3,1184,Viễn Đồng
4,1263,Miên Y Vệ


In [ ]:
dim_translator.head()

,translator_id,translator_name
0,1000010,Yurisa
1,1000034,Hoàng Châu
2,1000046,Huyền Linh
3,1000054,Nhuyễn Manh Đích Kelly
4,1000055,HacTamX


In [ ]:
dim_genre

,genre_id,genre_name
0,2,Tiên Hiệp
1,3,Huyền Huyễn
2,4,Khoa Huyễn
3,5,Võng Du
4,6,Đô Thị
5,7,Đồng Nhân
6,8,Dã Sử
7,9,Cạnh Kỹ
8,20,Kỳ Ảo


In [ ]:
# Lấy chuỗi ghi nhận ngày hiện tại
today = date.today()
formatted_date = today.strftime("%d-%m-%Y")

# Đặt tên file theo ngày lấy dữ liệu mới nhất
fact_book.to_excel("_".join(["Fact_Books", formatted_date]) + ".xlsx", index=False)
dim_author.to_excel("_".join(["Dim_Authors", formatted_date]) + ".xlsx", index=False)
dim_translator.to_excel("_".join(["Dim_Translators", formatted_date]) + ".xlsx", index=False)
dim_genre.to_excel("_".join(["Dim_Genres", formatted_date]) + ".xlsx", index=False)